In [3]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.5 MB/s eta 0:00:00


In [4]:
from datasets import load_dataset
HF_DATASET = "merionum/ru_paraphraser"
train_pd = load_dataset(HF_DATASET, split="train").to_pandas()
test_pd = load_dataset(HF_DATASET, split="test").to_pandas()

Generating train split:   0%|          | 0/7227 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1924 [00:00<?, ? examples/s]

In [5]:
from transformers import AutoTokenizer, AutoModel
import numpy as np
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer

bert_tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased-conversational")
bert_model = AutoModel.from_pretrained("DeepPavlov/rubert-base-cased-conversational").to("cuda")
bert_model.eval()

def get_bert_embeddings(
	text: str
) -> np.ndarray:
    tokens = bert_tokenizer.encode(
        text,
        add_special_tokens=True,
        truncation=True,
        max_length=512
    )
    n = 512  # max длина вектора
    padded = torch.LongTensor(
    	[
            np.array(tokens + [0] * (n - len(tokens)))
        ]
    )
    attention_mask = torch.LongTensor(
        np.where(
            padded != 0, 1, 0
        )
    )
    with torch.no_grad():
        batch_embeddings = bert_model(padded.to("cuda"), attention_mask=attention_mask.to("cuda"))[0][:, 0, :].cpu().numpy()
    return batch_embeddings

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased-conversational were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
labels = train_pd["class"].to_numpy()

In [7]:
train_pd

,id,id_1,id_2,text_1,text_2,class
0,1,201,8159,Полицейским разрешат стрелять на поражение по ...,Полиции могут разрешить стрелять по хулиганам ...,0
1,2,202,8158,Право полицейских на проникновение в жилище ре...,Правила внесудебного проникновения полицейских...,0
2,3,273,8167,Президент Египта ввел чрезвычайное положение в...,Власти Египта угрожают ввести в стране чрезвыч...,0
3,4,220,8160,Вернувшихся из Сирии россиян волнует вопрос тр...,Самолеты МЧС вывезут россиян из разрушенной Си...,-1
4,5,223,8160,В Москву из Сирии вернулись 2 самолета МЧС с р...,Самолеты МЧС вывезут россиян из разрушенной Си...,0
...,...,...,...,...,...,...
7222,25514,34622,34633,Путин освободил от должности ряд генералов,Путин снял с должностей более 20 руководителей...,0
7223,25524,34566,34654,Облака над Москвой в День Победы разгонят девя...,Путеводитель по Дню Победы: как провести 9 мая...,-1
7224,25548,34519,34681,Любляна отпразднует День Победы вместе с Москвой,В Москве ограничат движение в связи с Днем Победы,-1
7225,25549,34565,34681,Девять самолетов ВВС разгонят облака над Москв...,В Москве ограничат движение в связи с Днем Победы,-1


In [8]:
import tqdm

def get_data(pd):
    embeddings = []
    labels = []
    for index, row in tqdm.tqdm(pd.iterrows()):
        # Use BERT sentence separation token
        text = row["text_1"] + "[SEP]" + row["text_2"]
        embeddings.append(get_bert_embeddings(text))
        labels.append(int(int(row["class"])>-1))
    return embeddings, labels

In [9]:
train_embeddings, train_labels = get_data(train_pd)

0it [00:00, ?it/s]<ipython-input-5-33ca8f4cafe7>:20: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:274.)
  padded = torch.LongTensor(
7227it [03:52, 31.06it/s]


In [10]:
test_embeddings, test_labels = get_data(test_pd)

1924it [01:02, 31.03it/s]


In [11]:
train_embeddings = np.array(train_embeddings).squeeze()
test_embeddings = np.array(test_embeddings).squeeze()
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

In [12]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 4.0 MB/s eta 0:00:00


In [ ]:
import catboost
from catboost import CatBoostClassifier, Pool
cb_model = CatBoostClassifier(iterations=2000,
                           learning_rate=0.05,
                           depth=5,
                           loss_function='Logloss',
                           verbose=False)

# Fit model
cb_model.fit(train_embeddings, train_labels, cat_features=None)

In [ ]:
preds = cb_model.predict(test_embeddings)

# Evaluate the model
from sklearn.metrics import accuracy_score
print("Accuracy:", accuracy_score(test_labels, preds))

Accuracy: 0.7120582120582121


In [ ]:
from sklearn import metrics
f1_score_micro = metrics.f1_score(test_labels, preds, average='micro')
f1_score_macro = metrics.f1_score(test_labels, preds, average='macro')
print(f"Accuracy Score = {accuracy_score(test_labels, preds)}")
print(f"F1 Score (Micro) = {f1_score_micro}")
print(f"F1 Score (Macro) = {f1_score_macro}")

Accuracy Score = 0.7120582120582121
F1 Score (Micro) = 0.712058212058212
F1 Score (Macro) = 0.683056937567424
